# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

For this problem, we could do a chi-square to test the independence of the two populations proportions, that is,
call back rate and race. For this normality is not a requirement.

However, we could also to a z-test for equality of population proportions. The requirement for the z-test is that 
sample size * population proportion >= 5 for both populations. Yes, we can apply CLT and use z-test to test equality 
of two proportions.

The null hypothesis is that that call back rate for the both populations is the same, that is race has no effect
on the call back rate. The altermnate hypothesis is that call back rate is not the same.

In [3]:
# Permutation method to test equality of sample proportions
df = data[['call', 'race']]

df_w = df[df['race'] == 'w']
df_b = df[df['race'] == 'b']

# We draw permuted samples and then compute the replicates:

def permutation_sample(data1, data2):
    
    #Concatenate the data sets: data
    data = pd.concat([data1, data2], axis=0)
    
    # Permute the concatenated array: permuted_data
    permuted_data = np.random.permutation(data)

    #Split the permuted array into two: perm_sample_w, perm_sample_b
    perm_sample_1 = permuted_data[:len(data1)]
    perm_sample_2 = permuted_data[len(data1):]
    
    return perm_sample_1, perm_sample_2    
    
    
def draw_perm_reps(data1, data2, func, size=1):
    
    # Initialize array of replicates: perm_replicates
    perm_replicates = np.empty(size)
    p_1 = np.empty(size)
    p_2 = np.empty(size)
    
    for i in range(size):
        # Generate permutation sample
        perm_sample_1, perm_sample_2 = permutation_sample(data1, data2)

        # Compute the test statistic
        perm_replicates[i], p_1[i], p_2[i] = func(perm_sample_1, perm_sample_2)

    return perm_replicates, p_1, p_2


# Compute difference in proportions
def diff_proportions(data1, data2):
    

    p_1 = np.sum(data1) / len(data1)
    p_2 = np.sum(data2) / len(data2)
    
    diff_ratio = p_1 - p_2
    
    return diff_ratio, p_1, p_2

# Compute observed differnce
obs_diff, p_w_obs, p_b_obs = diff_proportions(df_w['call'], df_b['call'])

# Draw 10,000 permutation replicates: perm_replicates
perm_replicates, p_w, p_b = draw_perm_reps(df_w['call'], df_b['call'],
                                diff_proportions, size=10000)
              
        
# Compute p-value: p
p = np.sum(perm_replicates >= obs_diff)/len(perm_replicates)

print("p-value: {:.4f}".format(p))

p-value: 0.0000


The p-value is very small, so we reject the null hypothesis that there is no differnce in call back rates. 

In [4]:
#Compute 95% Confidence Interval of the proportion by Bootstrap

def draw_bs_reps(data, func, size=1):
    #Draw bootstrap replicates.

    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_replicates[i] = func(np.random.choice(data, 10000))
        

    return bs_replicates

# Define function 
def proportion(data):

    p = np.sum(data) / len(data)
    
    return p

bs_replicates_w = draw_bs_reps(df_w['call'], proportion, size=10000)
bs_replicates_b = draw_bs_reps(df_b['call'], proportion, size=10000)
diff_reps = bs_replicates_w - bs_replicates_b

#Compute 95% Confidence Interval of the proportion
conf_int_95_diff = np.percentile(diff_reps, [2.5, 97.5])

# Compute standard error
std_error = np.std(diff_reps)

print('95% Confidence Interval of the difference in call back rates: ', conf_int_95_diff)
print('Standard error of the difference in proportions is: {:.4f}'.format(std_error))


95% Confidence Interval of the difference in call back rates:  [ 0.0243  0.0396]
Standard error of the difference in proportions is: 0.0039


In [5]:
# Frequentist approach: Hypothesis test and Confidence Interval

df = data[['call', 'race']]

df_w = df[df['race'] == 'w']
df_b = df[df['race'] == 'b']

n_w = len(df_w['race'])
n_b = len(df_b['race'])

# Compute sample proportions: p_w, p_b
p_w = np.sum(df_w['call'] == 1) / n_w
p_b = np.sum(df_b['call'] == 1) / n_b

# Compute pooled "p" under the null hypothesis that Ho: p_w = p_b
p_bar = (n_w * p_w + n_b * p_w) / (n_w + n_b)

# Compute sample standard deviation of (p_w - p_b): std_p_wb
std_p_wb = np.sqrt(p_bar * (1-p_bar) * (1/n_w + 1/n_b))

# Compute the z test statistic: z
z = (p_w - p_b) / std_p_wb

print("z value: {:.4f}".format(z))

# Compute 95% Confidence rates by Frequentist method
# Compute Standard error of the difference in proportions
std_diff_prop = np.sqrt((p_w * (1-p_w)/n_w) + (p_b * (1-p_b)/n_w))

CI_low = (p_w - p_b) - 1.96 * std_diff_prop
CI_high = (p_w - p_b) + 1.96 * std_diff_prop

print("Standard error of differnce in proportions is: {:.4f}" .format(std_diff_prop))
print("Confidence Interval of difference in proportions is: [{:.4f}, {:.4f}]" .format(CI_low, CI_high))

z value: 3.7852
Standard error of differnce in proportions is: 0.0078
Confidence Interval of difference in proportions is: [0.0168, 0.0473]


z = 3.7852 > 2.58, we reject the null hypothesis at 1% level of significance. 

Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?
No. While our test did show that there is a statistical difference in call back rate based on race, there are other variables that may also contribute to, or interact with, race to explain the difference. Other factors, like education, age, gender could also affect the call back rate. 